介紹如何使用 *geopandas*
- https://www.cnblogs.com/feffery/p/13129271.html (中)
- https://zhuanlan.zhihu.com/p/345070554 (中)
- https://www.itread01.com/content/1581180364.html (中)
- https://geopandas.org/en/stable/getting_started/introduction.html (英)
- https://automating-gis-processes.github.io/CSC18/lessons/L2/geopandas-basics.html (英)

### Function

#### csv convert to shp and save [[link]](https://ithelp.ithome.com.tw/articles/10202839)

In [2]:

import pandas as pd 
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from collections import Counter

## import my modules
import sys
sys.path.append("..\\modules")
import convert
import aggregation

常用的座標分類：
 
- WMS,WMTS等常使用的Web Mercator :EPSG:3857
- kml, geojson默認使用的WGS84:EPSG:4326 (x = '經度', y = "緯度")
- TWD97 121分帶:EPSG:3826
- TWD97 119分帶:EPSG:3825
- TWD67 121分帶(台灣舊系統):EPSG:3828
- TWD67 119分帶(台灣舊系統):EPSG:3827

In [2]:
# Function for converting file (csv -> shp)
def Csv_to_Shp(dt,coor,crs_choise = {'init': 'epsg:3826'}): # 預設為TWD97 ，epsg:3826
    
    geom = [Point(xy) for xy in zip(dt[coor[0]], dt[coor[1]])]
    crs = crs_choise
    geodt = gpd.GeoDataFrame(dt, crs=crs, geometry=geom)
    
    return geodt

#### 計算區域內有幾個點、有多少人
- `geopandas.sjoin`
    - https://www.cnblogs.com/feffery/p/13129271.html
    - https://zhuanlan.zhihu.com/p/345070554 (中)
    - https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html

In [3]:

# Function for counting the point
# df_polygon.crs need to equal to df_point.crs

def HowManyPoint(df_polygon, df_point,x):  # x: gathering 的單位（里、區......）
    if df_polygon.crs != df_point.crs:
        df_polygon = df_polygon.to_crs(df_point.crs)
        
    aggr = gpd.sjoin(left_df=df_point ,
                right_df=df_polygon,
                op='intersects') \
            .groupby(x) \
            .size()
   
    return aggr


In [4]:
def HowMuch(df_polygon, df_point,x):  # x: gathering 的單位（里、區......）
    if df_polygon.crs != df_point.crs:
        df_polygon = df_polygon.to_crs(df_point.crs)
        
    aggr = gpd.sjoin(left_df=df_point ,
                right_df=df_polygon,
                op='intersects') \
            .groupby(x) \
            .sum()
    return aggr

### School
- https://www.tgos.tw/TGOS/Web/Metadata/TGOS_MetaData_View.aspx?MID=404530389BCC15967ED8D58E129BC680&SHOW_BACK_BUTTON=false)

因為學生數與教職員數有正相關，再加上私立的國高中並無完整的教職員資料，因此決定只採用學生數。

#### 國小

In [38]:
s = "Elementary"
'Roaming-Dogs-Data\School\109_Elementary\109_國小基本資料.csv'
path = "..\\..\\Roaming-Dogs-Data\\School\\109_Elementary"
df_01=pd.read_csv(path+"\\109學年度各級學校分布位置_國小.csv", encoding="utf-8")
print(df_01.columns)
df_01 = df_01.set_index("代碼")

df=pd.read_csv(path+"\\109_國小基本資料.csv", encoding="Big5")
print(df.columns)
df = df.set_index("學校代碼")
Elementary = pd.concat([df_01, df['學生數']], axis=1,join = "inner")
Elementary.shape
Elementary = Csv_to_Shp(dt = Elementary, coor = ['X 坐標', 'Y 坐標'])

Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")

Elementary = gpd.read_file(path+'\\109_Elementary_TWD97\\109_Elementary_TWD97.shp',encoding="Big5")
Elementary_list = HowManyPoint(df_polygon = Taiwan_village, df_point =Elementary ,x = 'VILLCODE')
Elementary_students = HowMuch(df_polygon = Taiwan_village, df_point =Elementary[["學生數",'geometry']] ,x = 'VILLCODE')
Elementary_list = pd.concat([Elementary_list, Elementary_students[["學生數"]]], axis=1)
Elementary_list.columns = [f"{s}", f"{s}_stu"]
print(Elementary_list.shape)

Elementary.to_file(driver='ESRI Shapefile', filename=path+'\\109_Elementary_TWD97\\109_Elementary_TWD97.shp', encoding="Big5", index = True)

Index(['學校級別', '代碼', '學校名稱', '縣市別', '鄉鎮市區', '郵遞區號', '地址', '電話', '網址', '體系別',
       '地區\n屬性', 'X 坐標', 'Y 坐標'],
      dtype='object')
Index(['縣市代碼', '縣市名稱', '鄉鎮市區', '學校代碼', '學校名稱', '1年級班級數', '2年級班級數', '3年級班級數',
       '4年級班級數', '5年級班級數', '6年級班級數', '1年級男學生數', '1年級女學生數', '2年級男學生數',
       '2年級女學生數', '3年級男學生數', '3年級女學生數', '4年級男學生數', '4年級女學生數', '5年級男學生數',
       '5年級女學生數', '6年級男學生數', '6年級女學生數', '學生數', '上學年男畢業生', '上學年女畢業生', '男專任教師',
       '女專任教師', '男職員', '女職員'],
      dtype='object')


C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)
C:\Users\ChenWenRu\anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
C:\Users\ChenWenRu\anaconda3\lib\site-packages\pyproj

(2581, 2)


#### 國中

In [ ]:
s = "Junior_high"
path = f"..\\..\\Roaming-Dogs-Data\\School\\109_{s}"
df_01=pd.read_csv(path+"\\109學年度各級學校分布位置_國中.csv", encoding="utf-8")
print(df_01.columns)
df_01 = df_01.set_index("代碼")

df=pd.read_csv(path+"\\109_國中基本資料.csv", encoding="Big5")
print(df.columns)
df = df.set_index("學校代碼")
Junior_high = pd.concat([df_01, df['學生數']], axis=1,join = "inner")
Junior_high = Csv_to_Shp(dt = Junior_high, coor = ['X 坐標', 'Y 坐標'])

Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")

Junior_high = gpd.read_file(path+f'\\109_{s}_TWD97\\109_{s}_TWD97.shp',encoding="Big5")
Junior_high_list = HowManyPoint(df_polygon = Taiwan_village, df_point =Junior_high ,x = 'VILLCODE')
Junior_high_students = HowMuch(df_polygon = Taiwan_village, df_point =Junior_high[["學生數",'geometry']] ,x = 'VILLCODE')
Junior_high_list = pd.concat([Junior_high_list, Junior_high_students[["學生數"]]], axis=1)
Junior_high_list.columns = [f"{s}", f"{s}_stu"]
Junior_high.to_file(driver='ESRI Shapefile', filename=path+f'\\109_{s}_TWD97\\109_{s}_TWD97.shp', encoding="Big5", index = True)

#### 高中職

In [ ]:
# 高中職
s = "Senior_high"
path = f"..\\..\\Roaming-Dogs-Data\\School\\109_{s}"
df_01=pd.read_csv(path+"\\109學年度各級學校分布位置_高中職.csv")
df_01 = df_01.set_index("代碼")
print(df_01.shape)

df=pd.read_csv(path+"\\109_高中基本資料.csv", encoding="Big5")
df = df.set_index("學校代碼")
df = df[["學生數"]].groupby(by=['學校代碼']).sum()
print(df.shape)

Senior_high = pd.concat([df_01, df['學生數']], axis=1,join = "inner")

Senior_high = Csv_to_Shp(dt = Senior_high, coor = ['X 坐標', 'Y 坐標'])

Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")
Senior_high_list = HowManyPoint(df_polygon = Taiwan_village, df_point =Senior_high ,x = 'VILLCODE')
Senior_high_students = HowMuch(df_polygon = Taiwan_village, df_point =Senior_high[["學生數",'geometry']] ,x = 'VILLCODE')
Senior_high_list = pd.concat([Senior_high_list, Senior_high_students[["學生數"]]], axis=1)
Senior_high_list.columns = [f"{s}", f"{s}_stu"]

Senior_high.to_file(driver='ESRI Shapefile', filename=path+f'\\109_{s}_TWD97\\109_{s}_TWD97.shp', encoding="Big5", index = True)

In [ ]:
# 學校
Taiwan_village_new = Taiwan_village[:]
Taiwan_village_new = Taiwan_village_new.set_index("VILLCODE")
Taiwan_village_new = pd.concat([Taiwan_village_new, Elementary_list], axis=1)
Taiwan_village_new = pd.concat([Taiwan_village_new, Junior_high_list], axis=1)
Taiwan_village_new = pd.concat([Taiwan_village_new, Senior_high_list], axis=1)
print(Taiwan_village_new.columns)

for i in [ 'Elementary','Elementary_stu', 'Junior_high', 'Junior_high_stu', 'Senior_high','Senior_high_stu']:
    Taiwan_village_new[i] = Taiwan_village_new[i].fillna(0)


# print(Taiwan_village_new.columns)
# print(Taiwan_village_new.head(10))
# print(Taiwan_village_new.shape)
Taiwan_village_new['VILLCODE'] = Taiwan_village_new.index
print(Taiwan_village_new.index)
Taiwan_village_new.to_file(driver='ESRI Shapefile', filename='..\\..\\Roaming-Dogs-Data\\School\\School\\School.shp')


### Hospital 
- https://sheethub.com/muyueh/%E5%85%A8%E5%8F%B0%E9%86%AB%E7%99%82%E9%99%A2%E6%89%80%E6%B8%85%E5%96%AE

- 特約類別：
    - "1"：醫學中心 (26)
    - "2"：區域醫院 (86)
    - "3"：地區醫院 (395)
    - "4"：診所 (6520)

In [ ]:
# 醫院、診所 

## 讀2個檔案
path = "..\\..\\Roaming-Dogs-Data\\Hospital\\Final"
df_01=pd.read_csv(path+"\\Hospital_A­_01_TWD97.csv", encoding="utf-8")
df_02=pd.read_csv(path+"\\Hospital_A_02_經緯度.csv", encoding="utf-8")

## 整理檔案
df_02 = df_02[(df_02["經度"] != 0) | (df_02["村里"] != "#VALUE!" )]
df_02_01 = df_02[df_02["經度"] != 0]
df_02_02 = df_02[df_02["經度"] == 0]

df_01_shp = Csv_to_Shp(dt = df_01,coor = ['X', 'Y'],crs_choise = {'init': 'epsg:3826'})
df_02_shp = Csv_to_Shp(dt = df_02_01,coor = ['經度', "緯度"],crs_choise = {'init': 'epsg:4326'})
df_02_shp = df_02_shp.to_crs(df_01_shp.crs)

df_02_shp.columns = df_01_shp.columns
# print(df_01_shp.columns)
col = ['醫事機構名稱', '機構縣市', '機構地址', '電話區域號碼', '電話號碼', '特約類別', '型態別', '醫事機構種類',
       '門牌號碼', '村里', 'geometry']
Hospital_geopd = pd.concat([df_01_shp[col], df_02_shp[col]], axis=0)

#print(Hospital_geopd[Hospital_geopd['特約類別'].isnull()]) # 找 nan 的位置
# 宜蘭縣羅東鎮南昌街83號
col_nan = ["電話區域號碼",'電話號碼','特約類別','型態別','醫事機構種類']
Hospital_geopd.loc[6677,col_nan] = Hospital_geopd.loc[6678,col_nan]

用特約類別分類(1、2、3、4)

In [ ]:
Hospital_geopd_1 = Hospital_geopd[Hospital_geopd['特約類別'] == 1]
print(Hospital_geopd_1.shape)
Hospital_geopd_2 = Hospital_geopd[Hospital_geopd['特約類別'] == 2]
print(Hospital_geopd_2.shape)
Hospital_geopd_3 = Hospital_geopd[Hospital_geopd['特約類別'] == 3]
print(Hospital_geopd_3.shape)
Hospital_geopd_4 = Hospital_geopd[Hospital_geopd['特約類別'] == 4]
print(Hospital_geopd_4.shape)



In [ ]:
Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")

Hospital_1_list = HowManyPoint(df_polygon = Taiwan_village, df_point =Hospital_geopd_1 ,x = 'VILLCODE')

Hospital_2_list = HowManyPoint(df_polygon = Taiwan_village, df_point =Hospital_geopd_2 ,x = 'VILLCODE')

Hospital_3_list = HowManyPoint(df_polygon = Taiwan_village, df_point =Hospital_geopd_3 ,x = 'VILLCODE')

Hospital_4_list = HowManyPoint(df_polygon = Taiwan_village, df_point =Hospital_geopd_4 ,x = 'VILLCODE')

Taiwan_village_new = Taiwan_village[:]
Taiwan_village_new = Taiwan_village_new.set_index("VILLCODE")
Taiwan_village_new = pd.concat([Taiwan_village_new, Hospital_1_list], axis=1)
col = list(Taiwan_village_new.columns)
col[-1] = 'Hospital_1'
Taiwan_village_new.columns = col

Taiwan_village_new = pd.concat([Taiwan_village_new, Hospital_2_list], axis=1)
col = list(Taiwan_village_new.columns)
col[-1] = 'Hospital_2'
Taiwan_village_new.columns = col

Taiwan_village_new = pd.concat([Taiwan_village_new, Hospital_3_list], axis=1)
col = list(Taiwan_village_new.columns)
col[-1] = 'Hospital_3'
Taiwan_village_new.columns = col

Taiwan_village_new = pd.concat([Taiwan_village_new, Hospital_4_list], axis=1)
col = list(Taiwan_village_new.columns)
col[-1] = 'Hospital_4'
Taiwan_village_new.columns = col
print(Taiwan_village_new.index)
Taiwan_village_new['VILLCODE'] = Taiwan_village_new.index

Taiwan_village_new.to_file(driver='ESRI Shapefile', \
    filename='..\\..\\Roaming-Dogs-Data\\Hospital\\Hospital\\Hospital.shp', encoding="Big5")

### Temple

[https://miao.temple01.com/tpl.php]

In [ ]:
# 讀檔
path = '..\\..\\Roaming-Dogs-Data\\Temple\\'
Temple_TWD97 = pd.read_csv(path+"Final\\Temple_TWD97_Village.csv")
Temple_TWD97 = Csv_to_Shp(dt= Temple_TWD97,coor = ['X','Y'])
Temple_lat = pd.read_csv(path+"Final\\Temple_經緯度_Village.csv")
Temple_lat = Csv_to_Shp(dt= Temple_lat,coor = ['經度','緯度'],crs_choise = {'init': 'epsg:4326'})

Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")

Temple_TWD97_list = HowManyPoint(df_polygon = Taiwan_village, df_point = Temple_TWD97,x = 'VILLCODE')
Temple_lat_list = HowManyPoint(df_polygon = Taiwan_village, df_point = Temple_lat,x = 'VILLCODE')



In [ ]:
# 2 個資料整理在一起
Taiwan_village = Taiwan_village.set_index("VILLCODE")
Taiwan_village_01 = pd.concat([Taiwan_village, Temple_TWD97_list, Temple_lat_list], axis=1)


In [ ]:
# only names

Temple_name = pd.read_csv(path+"Final\\Temple_Village.csv")

col = ['宗教團體名稱', '主管機關', '主祀神祇', '行政區',  'COUNTYNAME', 'TOWNNAME', 'VILLNAME', '地址', '教　別',
       '完整地址', 'X', 'Y']
Temple_name.columns = col
Temple_name_group = Temple_name.groupby(['TOWNNAME', 'VILLNAME']).size()
print(sum(Temple_name_group))

TOWN = [Temple_name_group.index[x][0] for x in range(len(Temple_name_group)) ]
VILL = [Temple_name_group.index[x][1] for x in range(len(Temple_name_group)) ]

data = {'TOWNNAME':TOWN, 'VILLNAME':VILL, 'Temple_02' : Temple_name_group.values}     
Temple_name = pd.DataFrame(data)  

Taiwan_village_Temple = Taiwan_village_01.merge(Temple_name, left_on = ["TOWNNAME","VILLNAME"], right_on = ["TOWNNAME","VILLNAME"], how = "left")
print(Temple_name.shape,Taiwan_village_Temple)
Taiwan_village_Temple.index = Taiwan_village_01.index


In [ ]:
for i in [0,1,'Temple_02']:
   Taiwan_village_Temple[i] =  Taiwan_village_Temple[i].fillna(0)
   
Taiwan_village_Temple['Temple'] =  Taiwan_village_Temple[[0,1,'Temple_02']]. sum(axis=1)
Taiwan_village_Temple['VILLCODE'] = (Taiwan_village_Temple.index)

col = ['VILLCODE', 'COUNTYNAME','TOWNNAME','VILLNAME','VILLENG','COUNTYID','COUNTYCODE','TOWNID','TOWNCODE',
      'NOTE','geometry','Temple']

Taiwan_village_Temple[col].to_file(driver='ESRI Shapefile', filename='..\\..\\Roaming-Dogs-Data\\Temple\\Temple\\Temple.shp', index = True)

### Train_station
- https://data.gov.tw/dataset/33425) (經緯度)
- https://data.gov.tw/dataset/8792 (人流) 2020 年整年

In [39]:
path = '..\\..\\Roaming-Dogs-Data\\Train_station\\'
station_people = pd.read_csv(path+'每日各站進出站人數20190423-20211231\\每日各站進出站人數2020.csv')

Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")
station_people.columns

C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


Index(['trnOpDate', 'staCode', 'gateInComingCnt', 'gateOutGoingCnt'], dtype='object')

In [41]:
# 進出站人數
station_people = station_people.set_index("staCode")
station_people["每日進出站平均人數"] = station_people[['gateInComingCnt', 'gateOutGoingCnt']].sum(axis=1)
print(station_people.columns)
station_people_mean = station_people[['每日進出站平均人數']].groupby("staCode").mean()


Index(['trnOpDate', 'gateInComingCnt', 'gateOutGoingCnt', '每日進出站平均人數'], dtype='object')


In [42]:
# 站數
station_gis = pd.read_csv(path+'車站基本資料集.csv', encoding="Utf-8")
station_gis = station_gis.set_index("stationCode")


In [43]:
station_gis.lon

stationCode
900     25.13401
910     25.12305
920     25.10843
930     25.09294
940     25.07803
          ...   
7360    25.10861
7361    25.13768
7362    25.13481
7380    25.10281
7390    25.10224
Name: lon, Length: 242, dtype: float64

In [44]:

station = pd.concat([station_people_mean, station_gis], axis=1,join = "outer")
idx = station['每日進出站平均人數'].isnull()
station = station[~idx]
station = Csv_to_Shp(dt= station,coor =  ["lat", "lon"],crs_choise = {'init': 'epsg:4326'})

station_list = HowManyPoint(df_polygon = Taiwan_village, df_point =station ,x = 'VILLCODE')
station_people = HowMuch(df_polygon = Taiwan_village, df_point =station[['每日進出站平均人數', 'geometry']] ,x = 'VILLCODE')


C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)
C:\Users\ChenWenRu\anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
C:\Users\ChenWenRu\anaconda3\lib\site-packages\pyproj

In [47]:


Taiwan_village = Taiwan_village.set_index("VILLCODE")
station_shp = pd.concat([Taiwan_village, station_list,station_people["每日進出站平均人數"]], axis=1)

In [48]:
station_shp['每日進出站平均人數']
sum(station_shp['每日進出站平均人數'].fillna(0))

1138216.0538818426

In [49]:
print(station_shp.shape, station_list)

col = ['COUNTYNAME',   'TOWNNAME',   'VILLNAME',    'VILLENG',   'COUNTYID',
       'COUNTYCODE',     'TOWNID',   'TOWNCODE',       'NOTE',   'geometry',
        "Train-station",  'Train-crowd']
station_shp.columns = col

for i in ["Train-station",  'Train-crowd']:
    station_shp[i] = station_shp[i].fillna(0)
    
station_shp['VILLCODE'] = station_shp.index
print(station_shp['VILLCODE'] )

station_shp.to_file(driver='ESRI Shapefile', \
    filename='..\\..\\Roaming-Dogs-Data\\Train_station\\Train_station\\Train_station.shp', encoding="Big5")

(7939, 12) VILLCODE
10002010045    1
10002020004    1
10002030008    1
10002030015    1
10002030017    1
              ..
68000020038    1
68000040001    1
68000040011    1
68000040020    1
68000040038    1
Length: 234, dtype: int64
VILLCODE
64000130006    64000130006
64000130008    64000130008
64000100010    64000100010
64000100013    64000100013
09007010006    09007010006
                  ...     
10008110015    10008110015
67000290019    67000290019
10008010030    10008010030
10008010010    10008010010
10008010029    10008010029
Name: VILLCODE, Length: 7939, dtype: object


### Tax 
- https://github.com/kiang/salary/blob/gh-pages/data/csv/2019.csv

In [ ]:
path = '..\\..\\Roaming-Dogs-Data\\Tax\\'
Tax = pd.read_csv(path+'109_Tax_02.csv')
print(Tax.columns)
col = ['縣市', '鄉鎮市區', '村里', 'TaxPayer', 'Tax_total', 'Tax_mean', 'Tax_median', 'Tax_Q1', 'Tax_Q3',
       'Tax_stander', 'Tax_CV']
Tax.columns = col
Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")
print(Taiwan_village.columns)
Taiwan_village_Tax = Taiwan_village.merge(Tax, left_on = ["TOWNNAME","VILLNAME"], right_on = ["鄉鎮市區","村里"], how = "left")

print(Tax.shape)
print(Taiwan_village.shape)
print(Taiwan_village_Tax.shape, Taiwan_village_Tax.columns)

Taiwan_village_Tax.to_file(driver='ESRI Shapefile', filename=path+'Tax\\Tax.shp', encoding="Big5", index = True)

In [ ]:
# idx = Taiwan_village_Tax['COUNTYNAME'].isnull()
# df_01 = Taiwan_village_Tax[['縣市','鄉鎮市區', '村里']][idx]
# print(Counter(df_01["縣市"]))
# print(df_01[df_01["縣市"] == '屏東縣'])
# idx = Taiwan_village_Tax['鄉鎮市區'].isnull()
# df_02 = Taiwan_village_Tax[['COUNTYNAME', 'TOWNNAME', 'VILLNAME']][idx]
# print((df_02[df_02["COUNTYNAME"] == '屏東縣'][['TOWNNAME', "VILLNAME"]]))

### 地價！
https://segis.moi.gov.tw/STAT/Web/Platform/Product/Apply/STAT_ProductApply.aspx?CD=DD0F83352C7C65BDC80ADF32B3BDC9EAAA4EED3EF54C3CF5&UTNAME=0D97A8168D7A7F9FA4D66682176BAA3E&ISOPEN=ON#

### Education
- https://data.gov.tw/dataset/8409

In [27]:
path = '..\\..\\Roaming-Dogs-Data\\Education\\'
Edu = pd.read_csv(path+'education_level.csv')
col = [ 'district_code','high_rat', 'mid_rat', 'low_rat','high_edu', 'mid_edu', 'low_edu']
Edu = Edu[col]
Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")
Taiwan_village_Edu = Taiwan_village.merge(Edu, left_on = "VILLCODE", right_on = "district_code", how = "left")
Taiwan_village_Edu = gpd.GeoDataFrame(Taiwan_village_Edu, geometry = 'geometry')
Taiwan_village_Edu = Taiwan_village_Edu.drop('district_code', axis = 1)
Taiwan_village_Edu.to_file(driver='ESRI Shapefile', filename=path+'Education\\Education.shp', encoding="Big5")


In [28]:
Taiwan_village_Edu.head(1)

,VILLCODE,COUNTYNAME,TOWNNAME,VILLNAME,VILLENG,COUNTYID,COUNTYCODE,TOWNID,TOWNCODE,NOTE,geometry,high_rat,mid_rat,low_rat,high_edu,mid_edu,low_edu
0,64000130006,高雄市,林園區,中門里,Zhongmen Vil.,E,64000,E13,64000130,None,"POLYGON ((120.36772 22.49564, 120.36427 22.498...",0.188906129,0.406904577,0.404189294,487,1049,1042


### Population 109/09
- https://segis.moi.gov.tw/STAT/Web/Platform/Product/Apply/STAT_ProductApply.aspx?SECTION=069F0C77479B8EB9&THEM=16EB5C58C90A28399BB172E2EA27B64F


In [ ]:

path = '..\\..\\Roaming-Dogs-Data\\Population\\'

Taiwan_village_pop_01 = pd.read_csv(path+ '109年9月行政區人口指標_村里\\109年9月行政區人口指標_村里.csv')
print(Taiwan_village_pop_01.shape)
Taiwan_village_pop_01 = Taiwan_village_pop_01.drop(0, axis=0)
print(Taiwan_village_pop_01.shape)


- COUNTY_ID	縣市代碼
- COUNTY	縣市名稱
- TOWN_ID	鄉鎮市區代碼
- TOWN	鄉鎮市區名稱
- V_ID	村里代碼
- VILLAGE	村里名稱
- M_F_RAT	性比例
- P_H_CNT	戶量
- P_DEN	人口密度
- DEPENDENCY_RAT	扶養比
- A0A14_A15A65_RAT	扶幼比
- A65UP_A15A64_RAT	扶老比
- A65_A0A14_RAT	老化指數
- INFO_TIME	資料時間


In [ ]:

Taiwan_village_pop_02 = pd.read_csv(path+ '109年9月行政區人口統計_村里\\109年9月行政區人口統計_村里.csv')
print(Taiwan_village_pop_02.shape)
Taiwan_village_pop_02 = Taiwan_village_pop_02.drop(0, axis=0)
print(Taiwan_village_pop_02.shape)


- COUNTY_ID	縣市代碼
- COUNTY	縣市名稱
- TOWN_ID	鄉鎮市區代碼
- TOWN	鄉鎮市區名稱
- V_ID	村里代碼
- VILLAGE	村里名稱
- H_CNT	戶數
- P_CNT	人口數
- M_CNT	男性人口數
- F_CNT	女性人口數
- INFO_TIME	資料時間



In [ ]:
print(Taiwan_village_pop_01.columns)
print(Taiwan_village_pop_02.columns)

Taiwan_village_pop = Taiwan_village_pop_01.merge(Taiwan_village_pop_02, \
    left_on =['COUNTY', 'TOWN', 'VILLAGE', 'COUNTY_ID', 'TOWN_ID', 'V_ID','VILLAGE'],\
        right_on = ['COUNTY', 'TOWN', 'VILLAGE', 'COUNTY_ID', 'TOWN_ID', 'V_ID','VILLAGE'], how = "outer")
print(Taiwan_village_pop_01.shape, Taiwan_village_pop_02.shape, Taiwan_village_pop.shape,Taiwan_village_pop.columns)
idx = Taiwan_village_pop["V_ID"].isnull()
Taiwan_village_pop = Taiwan_village_pop[~idx]

Taiwan_village_pop["V_ID"] = [x.replace( '-','' ) for x in Taiwan_village_pop["V_ID"]]
print(Taiwan_village_pop.shape)

In [ ]:
print(Taiwan_village_pop.V_ID)

In [ ]:
col = ['COUNTY_ID', 'COUNTY', 'TOWN_ID', 'TOWN', 'V_ID', 'VILLAGE', 'M_F_RAT',\
       'P_H_CNT', 'P_DEN', 'DEPENDENCY_RAT', 'A0A14_A15A65_RAT',\
       'A65UP_A15A64_RAT', 'A65_A0A14_RAT',  'H_CNT', 'P_CNT', 'M_CNT', 'F_CNT']
# Taiwan_village_pop = Taiwan_village_pop.set_index("V_ID")
Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")
print(Taiwan_village.crs)
# print(type(Taiwan_village["VILLCODE"][0]))
# Taiwan_village = Taiwan_village.set_index("VILLCODE")
Taiwan_village_pop = Taiwan_village.merge( Taiwan_village_pop[col], left_on = 'VILLCODE', right_on = "V_ID", how = "left")
Taiwan_village_pop.columns

In [ ]:
col = ['VILLCODE', 'COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry', 'VILLAGE', 'M_F_RAT', 'P_H_CNT',
       'P_DEN', 'DEPENDENCY_RAT', 'A0A14_A15A65_RAT', 'A65UP_A15A64_RAT',
       'A65_A0A14_RAT', 'H_CNT', 'P_CNT', 'M_CNT', 'F_CNT']
Taiwan_village_pop = gpd.GeoDataFrame(Taiwan_village_pop[col], geometry = 'geometry')


In [ ]:
Taiwan_village_pop.to_file(driver='ESRI Shapefile', \
   filename=path+'Population\\Population.shp',encoding = 'big5')

### Clustering

延續 107 年度調查之分層方法，將母體依縣市分割為 22 個次母體，再將各次母體縣市中的鄉、鎮、市、區以**人口密度**、**農業人口比例**、**專科以上人口比例**及**老年人口比例**等，以群分析作為都市化程度分層依據分為二至四層

#### 分群結果
    1. 於該縣市中，都市戶程度較高的鄉鎮市區
    2. 於該縣市中，都市戶程度中等的鄉鎮市區
    3. 於該縣市中，都市戶程度較低的鄉鎮市區

- 宜蘭縣
	1.	羅東鎮、宜蘭市
	2.	五結鄉、冬山鄉、蘇澳鎮、礁溪鄉、頭城鎮、壯圍鄉、員山鄉、三星鄉、大同鄉、南澳鄉
- 花蓮縣	
	1.	花蓮市
	2.	吉安鄉、新城鄉、玉里鎮、鳳林鎮、瑞穗鄉、壽豐鄉、光復鄉、富里鄉、豐濱鄉、秀林鄉、萬榮鄉、卓溪鄉
- 金門縣	
	1.	金城鎮、金寧鎮、烈嶼鄉、金湖鎮、烏坵鄉、金沙鎮
- 南投縣
	1.	南投市、草屯鎮、名間鄉、埔里鎮、竹山鎮
	2.	集集鎮、水里鄉、魚池鄉、鹿谷鄉、國姓鄉、中寮鄉、仁愛鄉、信義鄉
- 屏東縣	
	1.	東港鎮、潮州鎮、恆春鎮
	2.	屏東市
	3.	新園鄉、萬丹鄉、長治鄉、麟洛鄉、內埔鄉、鹽埔鄉、里港鄉、九如鄉、崁頂鄉、林邊鄉、枋寮鄉、佳冬鄉、竹田鄉、南州鄉、萬巒鄉、枋山鄉、高樹鄉、車城鄉、新埤鄉、瑪家鄉、來義鄉、泰武鄉、三地門鄉、滿州鄉、春日鄉、牡丹鄉、獅子鄉、霧臺鄉
- 苗栗縣
	1.	苗栗市、竹南鎮、頭份市、苑裡鎮
	2.	公館鄉、造橋鄉、三義鄉、銅鑼鄉、頭屋鄉、後龍鎮、通霄鎮、卓蘭鎮、西湖鄉、大湖鄉、三灣鄉、南庄鄉、獅潭鄉、泰安鄉
- 桃園市
	1.	桃園區、平鎮區、八德區、中壢區
	2.	龜山區、蘆竹區、楊梅區、龍潭區、大溪區、大園區、觀音區、新屋區、復興區
- 高雄市
	1.	新興區、苓雅區、旗津區、三民區、鹽埕區、前金區、鳳山區、左營區、前鎮區、鼓山區、楠梓區
	2.	小港區、梓官區、仁武區、林園區、岡山區、茄萣區、鳥松區、湖內區、橋頭區、大寮區、彌陀區、大社區、路竹區、阿蓮區、大樹區、燕巢區、永安區
	3.	旗山區、美濃區、內門區、杉林區、田寮區、甲仙區、六龜區、那瑪夏區、茂林區、桃源區
- 基隆市
	1.	仁愛區、中山區、中正區、信義區、安樂區、暖暖區、七堵區
- 連江縣
	1.	南竿鄉、莒光鄉、東引鄉、北竿鄉
- 雲林縣
	1.	斗六市、虎尾鎮、北港鎮、斗南鎮、西螺鎮、林內鄉、土庫鎮、麥寮鄉、莿桐鄉
	2.	臺西鄉、二崙鄉、大埤鄉、崙背鄉、口湖鄉、元長鄉、水林鄉、褒忠鄉、東勢鄉、四湖鄉、古坑鄉
- 新北市
	1.	永和區、蘆洲區、板橋區、三重區、新莊區、中和區
	2.	土城區、泰山區、鶯歌區、樹林區、汐止區、五股區、淡水區、新店區、林口區、深坑區、八里區、三峽區
	3.	金山區、瑞芳區、三芝區、萬里區、石門區、烏來區、貢寮區、平溪區、雙溪區、坪林區、石碇區
- 新竹市
	1.	北區、東區、香山區
- 新竹縣
	1.	竹北市、竹東鎮、湖口鄉、新豐鄉
	2.	芎林鄉、新埔鎮、寶山鄉、關西鎮、橫山鄉、北埔鄉、峨眉鄉、五峰鄉、尖石鄉
- 嘉義市
	1.	西區、東區
- 嘉義縣
	1.	民雄鄉、朴子市、水上鄉、大林鎮、新港鄉、中埔鄉、太保市
	2.	溪口鄉、布袋鎮、六腳鄉、東石鄉、鹿草鄉、義竹鄉、竹崎鄉、梅山鄉、番路鄉、大埔鄉、阿里山鄉
- 彰化縣
	1.	彰化市、員林市、和美鎮、鹿港鎮、永靖鄉、溪湖鎮、北斗鎮、埔心鄉、伸港鄉、秀水鄉、花壇鄉、田中鎮、大村鄉、社頭鄉
	2.	田尾鄉、福興鄉、線西鄉、埔鹽鄉、埤頭鄉、芬園鄉、溪州鄉、二林鎮、芳苑鄉、竹塘鄉、二水鄉、大城鄉
- 臺中市
	1.	北區、中區、西區、南區、東區、大里區、北屯區、西屯區、南屯區
	2.	潭子區、豐原區、梧棲區、大雅區、沙鹿區、龍井區、神岡區、烏日區、大肚區、清水區、太平區、大甲區、后里區、外埔區、霧峰區、石岡區、大安區、東勢區、新社區、和平區
- 臺北市
	1.	大安區、大同區、松山區、萬華區、中正區、信義區、中山區、內湖區、文山區、士林區、北投區、南港區
- 臺東縣
	1.	臺東市
	2.	綠島鄉、關山鎮、太麻里鄉、池上鄉、成功鎮、鹿野鄉、長濱鄉、東河鄉、蘭嶼鄉、大武鄉、卑南鄉、達仁鄉、金峰鄉、延平鄉、海端鄉
- 臺南市
	1.	東區、北區、中西區、南區、安平區、永康區
	2.	新營區、安南區、仁德區、佳里區、歸仁區、善化區、新市區、新化區
	3.	安定區、麻豆區、西港區、下營區、鹽水區、學甲區、關廟區、柳營區、六甲區、官田區、山上區、北門區、七股區
	4.	將軍區、後壁區、白河區、玉井區、東山區、大內區、楠西區、左鎮區、龍崎區、南化區
- 澎湖縣
	1.	馬公市、七美鄉、白沙鄉、西嶼鄉、湖西鄉、望安鄉

 


In [ ]:
YLA = ["宜蘭縣"]
YLA_01= ["羅東鎮","宜蘭市"]
YLA_02 = ["五結鄉","冬山鄉","蘇澳鎮","礁溪鄉","頭城鎮","壯圍鄉","員山鄉","三星鄉","大同鄉","南澳鄉"]

HL= ["花蓮縣"]	
HL_01 = ["花蓮市"]
HL_02 = ["吉安鄉","新城鄉","玉里鎮","鳳林鎮","瑞穗鄉","壽豐鄉","光復鄉","富里鄉","豐濱鄉","秀林鄉","萬榮鄉","卓溪鄉"]

JM = ["金門縣"]	
JM_01 = ["金城鎮","金寧鄉","烈嶼鄉","金湖鎮","烏坵鄉","金沙鎮"]

NT = ["南投縣"]
NT_01 = ["南投市","草屯鎮","名間鄉","埔里鎮","竹山鎮"]
NT_02 = ["集集鎮","水里鄉","魚池鄉","鹿谷鄉","國姓鄉","中寮鄉","仁愛鄉","信義鄉"]

PD = ["屏東縣"]	
PD_01 = ["東港鎮","潮州鎮","恆春鎮"]
PD_02 = ["屏東市"]
PD_03 = ["新園鄉","萬丹鄉","長治鄉","麟洛鄉","內埔鄉","鹽埔鄉","里港鄉","九如鄉","崁頂鄉","林邊鄉","枋寮鄉","佳冬鄉","竹田鄉","南州鄉","萬巒鄉","枋山鄉","高樹鄉","車城鄉","新埤鄉","瑪家鄉","來義鄉","泰武鄉","三地門鄉","滿州鄉","春日鄉","牡丹鄉","獅子鄉","霧臺鄉"]

ML = ["苗栗縣"]
ML_01 = ["苗栗市","竹南鎮","頭份市","苑裡鎮"]
ML_02 = ["公館鄉","造橋鄉","三義鄉","銅鑼鄉","頭屋鄉","後龍鎮","通霄鎮","卓蘭鎮","西湖鄉","大湖鄉","三灣鄉","南庄鄉","獅潭鄉","泰安鄉"]

TY = ["桃園市"]
TY_01 = ["桃園區","平鎮區","八德區","中壢區"]
TY_02 = ["龜山區","蘆竹區","楊梅區","龍潭區","大溪區","大園區","觀音區","新屋區","復興區"]

KS = ["高雄市"]
KS_01 = ["新興區","苓雅區","旗津區","三民區","鹽埕區","前金區","鳳山區","左營區","前鎮區","鼓山區","楠梓區"]
KS_02 = ["小港區","梓官區","仁武區","林園區","岡山區","茄萣區","鳥松區","湖內區","橋頭區","大寮區","彌陀區","大社區","路竹區","阿蓮區","大樹區","燕巢區","永安區"]
KS_03 = ["旗山區","美濃區","內門區","杉林區","田寮區","甲仙區","六龜區","那瑪夏區","茂林區","桃源區"]

JL = ["基隆市"]
JL_01 = ["仁愛區","中山區","中正區","信義區","安樂區","暖暖區","七堵區"]

LJ = ["連江縣"]
LJ_01 = ["南竿鄉","莒光鄉","東引鄉","北竿鄉"]

YLI = ["雲林縣"]
YLI_01 = ["斗六市","虎尾鎮","北港鎮","斗南鎮","西螺鎮","林內鄉","土庫鎮","麥寮鄉","莿桐鄉"]
YLI_02 = ["臺西鄉","二崙鄉","大埤鄉","崙背鄉","口湖鄉","元長鄉","水林鄉","褒忠鄉","東勢鄉","四湖鄉","古坑鄉"]

SP = ["新北市"]
SP_01 = ["永和區","蘆洲區","板橋區","三重區","新莊區","中和區"]
SP_02 = ["土城區","泰山區","鶯歌區","樹林區","汐止區","五股區","淡水區","新店區","林口區","深坑區","八里區","三峽區"]
SP_03 = ["金山區","瑞芳區","三芝區","萬里區","石門區","烏來區","貢寮區","平溪區","雙溪區","坪林區","石碇區"]

SJSHIH = ["新竹市"]
SJSHIH_01 = ["北區","東區","香山區"]

SJSHIAN = ["新竹縣"]
SJSHIAN_01 =["竹北市","竹東鎮","湖口鄉","新豐鄉"]
SJSHIAN_02 = ["芎林鄉","新埔鎮","寶山鄉","關西鎮","橫山鄉","北埔鄉","峨眉鄉","五峰鄉","尖石鄉"]

JYSHIN = ["嘉義市"]
JYSHIN_01 = ["西區","東區"]

JYSHIAN = ["嘉義縣"]
JYSHIAN_01 = ["民雄鄉","朴子市","水上鄉","大林鎮","新港鄉","中埔鄉","太保市"]
JYSHIAN_02 = ["溪口鄉","布袋鎮","六腳鄉","東石鄉","鹿草鄉","義竹鄉","竹崎鄉","梅山鄉","番路鄉","大埔鄉","阿里山鄉"]

JH = ["彰化縣"]
JH_01 = ["彰化市","員林市","和美鎮","鹿港鎮","永靖鄉","溪湖鎮","北斗鎮","埔心鄉","伸港鄉","秀水鄉","花壇鄉","田中鎮","大村鄉","社頭鄉"]
JH_02 = ["田尾鄉","福興鄉","線西鄉","埔鹽鄉","埤頭鄉","芬園鄉","溪州鄉","二林鎮","芳苑鄉","竹塘鄉","二水鄉","大城鄉"]

TC = ["臺中市"]
TC_01 = ["北區","中區","西區","南區","東區","大里區","北屯區","西屯區","南屯區"]
TC_02 = ["潭子區","豐原區","梧棲區","大雅區","沙鹿區","龍井區","神岡區","烏日區","大肚區","清水區","太平區","大甲區","后里區","外埔區","霧峰區","石岡區","大安區","東勢區","新社區","和平區"]

TP = ["臺北市"]
TP_01 = ["大安區","大同區","松山區","萬華區","中正區","信義區","中山區","內湖區","文山區","士林區","北投區","南港區"]

TD = ["臺東縣"]
TD_01 = ["臺東市"]
TD_02 = ["綠島鄉","關山鎮","太麻里鄉","池上鄉","成功鎮","鹿野鄉","長濱鄉","東河鄉","蘭嶼鄉","大武鄉","卑南鄉","達仁鄉","金峰鄉","延平鄉","海端鄉","琉球鄉"]

TN = ["臺南市"]
TN_01 = ["東區","北區","中西區","南區","安平區","永康區"]
TN_02 = ["新營區","安南區","仁德區","佳里區","歸仁區","善化區","新市區","新化區"]
TN_03 = ["安定區","麻豆區","西港區","下營區","鹽水區","學甲區","關廟區","柳營區","六甲區","官田區","山上區","北門區","七股區"]
TN_04 = ["將軍區","後壁區","白河區","玉井區","東山區","大內區","楠西區","左鎮區","龍崎區","南化區"]

PH = ["澎湖縣"]
PH_01 = ["馬公市","七美鄉","白沙鄉","西嶼鄉","湖西鄉","望安鄉"]

In [ ]:
CLU_01 = YLA_01+HL_01+JM_01+NT_01+PD_01+ML_01+TY_01+KS_01+JL_01+LJ_01\
    +YLI_01+SP_01+SJSHIH_01+SJSHIAN_01+JYSHIN_01+JYSHIAN_01+JH_01+TC_01\
        +TP_01+TD_01+TN_01+PH_01
        
CLU_02 = YLA_02+HL_02+NT_02+PD_02+ML_02+TY_02+KS_02\
    +YLI_02+SP_02+SJSHIAN_02+JYSHIAN_02+JH_02+TC_02\
        +TD_02+TN_02

CLU_03 = PD_03+KS_03+SP_03+TN_03

CLU_04 = TN_04

In [ ]:
Taiwan_village_clu = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")
CLU = []

for x in Taiwan_village_clu["TOWNNAME"]:
    if x in CLU_01:
        CLU.append(1)
    elif x in CLU_02:
        CLU.append(2)
    elif x in CLU_03:
        CLU.append(3)
    elif x in CLU_04:
        CLU.append(4)
    else:
        CLU.append(np.nan) 
Taiwan_village_clu["Cluster"] = CLU
Taiwan_village_clu.to_file("..\\..\\Roaming-Dogs-Data\\Cluster\\Cluster.shp",encoding="Big5")


### Combined all data

In [37]:
Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")
TW_Clu = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Cluster\\Cluster.shp",encoding="big5")
TW_Clu = TW_Clu.drop(labels=['COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',\
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'], axis=1)
TW_Edu = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Education\\Education\\Education.shp",encoding="big5")
TW_Edu = TW_Edu.drop(labels=['COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',\
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'], axis=1)
TW_Hospital = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Hospital\\Hospital\\Hospital.shp",encoding="big5")
TW_Hospital = TW_Hospital.drop(labels=['COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',\
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'], axis=1)
TW_Population = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Population\\Population\\Population.shp",encoding="big5")
TW_Population = TW_Population.drop(labels=['COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',\
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'], axis=1)
TW_School = gpd.read_file("..\\..\\Roaming-Dogs-Data\\School\\School\\School.shp",encoding="big5")
TW_School = TW_School.drop(labels=['COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',\
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'], axis=1)
TW_Tax = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Tax\\Tax\\Tax.shp",encoding="big5")
TW_Tax = TW_Tax.drop(labels=['COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID','縣市', '鄉鎮市區', '村里',\
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'], axis=1)
TW_Temple = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Temple\\Temple\\Temple.shp",encoding="big5")
TW_Temple = TW_Temple.drop(labels=['COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',\
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'], axis=1)
TW_Train = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Train_station\\Train_station\\Train_station.shp",encoding="big5")
TW_Train = TW_Train.drop(labels=['COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',\
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'], axis=1)

In [38]:
n = 0
print(n,Taiwan_village.columns,Taiwan_village.shape )
n +=1
print(n,TW_Clu.columns,Taiwan_village.shape )
n +=1
print(n,TW_Edu.columns,Taiwan_village.shape )
n +=1
print(n,TW_Hospital.columns,TW_Hospital.shape)
n +=1
print(n,TW_Population.columns,TW_Population.shape)
n +=1
print(n,TW_School.columns,TW_School.shape)
n +=1
print(n,TW_Tax.columns,TW_Tax.shape)
n +=1
print(n,TW_Temple.columns,TW_Temple.shape)
n +=1
print(n,TW_Train.columns,TW_Train.shape)

0 Index(['VILLCODE', 'COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry'],
      dtype='object') (7939, 11)
1 Index(['VILLCODE', 'Cluster'], dtype='object') (7939, 11)
2 Index(['VILLCODE', 'high_rat', 'mid_rat', 'low_rat', 'high_edu', 'mid_edu',
       'low_edu'],
      dtype='object') (7939, 11)
3 Index(['Hospital_1', 'Hospital_2', 'Hospital_3', 'Hospital_4', 'VILLCODE'], dtype='object') (7939, 5)
4 Index(['VILLCODE', 'VILLAGE', 'M_F_RAT', 'P_H_CNT', 'P_DEN', 'DEPENDENCY',
       'A0A14_A15A', 'A65UP_A15A', 'A65_A0A14_', 'H_CNT', 'P_CNT', 'M_CNT',
       'F_CNT'],
      dtype='object') (7939, 13)
5 Index(['Elementary', 'Elementa_1', 'Junior_hig', 'Junior_h_1', 'Senior_hig',
       'Senior_h_1', 'VILLCODE'],
      dtype='object') (7939, 7)
6 Index(['VILLCODE', 'TaxPayer', 'Tax_total', 'Tax_mean', 'Tax_median', 'Tax_Q1',
       'Tax_Q3', 'Tax_stande', 'Tax_CV'],
      dtype='object') (7967, 9)
7 Index(['VILLCODE', 'T

In [46]:
Taiwan_village = gpd.read_file("..\\..\\Roaming-Dogs-Data\\Taiwan_village\\VILLAGE_MOI_1101214.shp",encoding="utf-8")
Taiwan_village_02 = Taiwan_village.to_crs("EPSG:3826")
print(Taiwan_village.area,Taiwan_village_02.area,Taiwan_village_02 )

C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


0       0.000184
1       0.000115
2       0.000099
3       0.000233
4       0.000101
          ...   
7934    0.001268
7935    0.000376
7936    0.000202
7937    0.000140
7938    0.000403
Length: 7939, dtype: float64 0       2.101052e+06
1       1.306214e+06
2       1.130898e+06
3       2.653210e+06
4       1.117617e+06
            ...     
7934    1.431354e+07
7935    4.268018e+06
7936    2.278451e+06
7937    1.581437e+06
7938    4.541425e+06
Length: 7939, dtype: float64          VILLCODE COUNTYNAME TOWNNAME VILLNAME         VILLENG COUNTYID  \
0     64000130006        高雄市      林園區      中門里   Zhongmen Vil.        E   
1     64000130008        高雄市      林園區      港埔里     Gangpu Vil.        E   
2     64000100010        高雄市      旗津區      上竹里   Shangzhu Vil.        E   
3     64000100013        高雄市      旗津區      中興里  Zhongxing Vil.        E   
4     09007010006        連江縣      南竿鄉      津沙村     Jinsha Vil.        Z   
...           ...        ...      ...      ...             ...      ...   

C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [25]:
Taiwan_village_ = Taiwan_village[:]
Taiwan_village_  = Taiwan_village_.merge(TW_Clu, left_on = ['VILLCODE'], right_on = ['VILLCODE'])
Taiwan_village_  = Taiwan_village_.merge(TW_Edu, left_on = ['VILLCODE'], right_on = ['VILLCODE'])
Taiwan_village_  = Taiwan_village_.merge(TW_Hospital, left_on = ['VILLCODE'], right_on = ['VILLCODE'])
Taiwan_village_  = Taiwan_village_.merge(TW_Population, left_on = ['VILLCODE'], right_on = ['VILLCODE'])
Taiwan_village_  = Taiwan_village_.merge(TW_School, left_on = ['VILLCODE'], right_on = ['VILLCODE'])
Taiwan_village_  = Taiwan_village_.merge(TW_Tax, left_on = ['VILLCODE'], right_on = ['VILLCODE'])
Taiwan_village_  = Taiwan_village_.merge(TW_Temple, left_on = ['VILLCODE'], right_on = ['VILLCODE'])
Taiwan_village_  = Taiwan_village_.merge(TW_Train,left_on = ['VILLCODE'], right_on = ['VILLCODE'])
Taiwan_village_['Hospital'] = Taiwan_village_[['Hospital_1','Hospital_2','Hospital_3']].sum(axis=1)
Taiwan_village_['Clinic'] = Taiwan_village_['Hospital_4']
print(Taiwan_village_.shape)
print(Taiwan_village_.columns, len(Taiwan_village_.columns))


(7967, 53)
Index(['VILLCODE', 'COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry', 'Cluster',
       'high_rat', 'mid_rat', 'low_rat', 'high_edu', 'mid_edu', 'low_edu',
       'Hospital_1', 'Hospital_2', 'Hospital_3', 'Hospital_4', 'VILLAGE',
       'M_F_RAT', 'P_H_CNT', 'P_DEN', 'DEPENDENCY', 'A0A14_A15A', 'A65UP_A15A',
       'A65_A0A14_', 'H_CNT', 'P_CNT', 'M_CNT', 'F_CNT', 'Elementary',
       'Elementa_1', 'Junior_hig', 'Junior_h_1', 'Senior_hig', 'Senior_h_1',
       'TaxPayer', 'Tax_total', 'Tax_mean', 'Tax_median', 'Tax_Q1', 'Tax_Q3',
       'Tax_stande', 'Tax_CV', 'Temple', 'Train-stat', 'Train-crow',
       'Hospital', 'Clinic'],
      dtype='object') 53


In [8]:
Taiwan_village_.columns

Index(['VILLCODE', 'COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry', 'Cluster',
       'high_rat', 'mid_rat', 'low_rat', 'high_edu', 'mid_edu', 'low_edu',
       'Hospital_1', 'Hospital_2', 'Hospital_3', 'Hospital_4', 'VILLAGE',
       'M_F_RAT', 'P_H_CNT', 'P_DEN', 'DEPENDENCY', 'A0A14_A15A', 'A65UP_A15A',
       'A65_A0A14_', 'H_CNT', 'P_CNT', 'M_CNT', 'F_CNT', 'Elementary',
       'Elementa_1', 'Junior_hig', 'Junior_h_1', 'Senior_hig', 'Senior_h_1',
       'TaxPayer', 'Tax_total', 'Tax_mean', 'Tax_median', 'Tax_Q1', 'Tax_Q3',
       'Tax_stande', 'Tax_CV', 'Temple', 'Train-stat', 'Train-crow'],
      dtype='object')

In [26]:
col = ['VILLCODE', 'COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'geometry', 'Cluster',
       'high_rat', 'mid_rat', 'low_rat', 'high_edu', 'mid_edu', 'low_edu',
       'Hospital_1', 'Hospital_2', 'Hospital_3', 'Hospital_4', 'VILLAGE',
       'M_F_RAT', 'P_H_CNT', 'P_DEN', 'DEPENDENCY', 'YOUN_DEP', 'OLD_DEP', 
       'AGING_IDX', 'H_CNT', 'P_CNT', 'M_CNT', 'F_CNT', 'Ele', 
       'Ele_stu', 'Junior', 'Junior_stu', 'Senior', 'Senior_stu', 
       'TaxPayer', 'Income_total','Income_mean', 'Income_median', 'Income_Q1',
       'Income_Q3', 'Income_stande', 'Income_CV',  'Temple', 'Train-stat',
       'Train-crow','Hospital', 'Clinic']
print(len(col))
Taiwan_village_.columns = col
Taiwan_village_ = Taiwan_village_.drop('VILLAGE', axis = 1)

53


In [27]:

Taiwan_village_ = gpd.GeoDataFrame(Taiwan_village_, geometry = 'geometry')
Taiwan_village_.crs = Taiwan_village.crs


In [28]:
Taiwan_village_["Area"] = Taiwan_village_.area


In [29]:
Taiwan_village_.to_csv('..\\..\\Roaming-Dogs-Data\@Taiwan_village_variable\\Taiwan_village_variable.csv', encoding='utf-8')

#### CodeBook

In [31]:
col_explain= ['縣市名稱', "鄉鎮市區名稱",'村里名稱', '村里英文' , "縣市ID",\
            '縣市代碼', "鄉鎮市區ID", '鄉鎮市區代碼','村里代碼', '備註','分群',\
            '高等教育人口比','中等教育人口比','初等教育人口比',\
            '高等教育人口','中等教育人口','初等教育人口','醫學中心', '區域醫院', '地區醫院','診所', '性比例',\
            '戶量','人口密度','扶養比','扶幼比','扶老比',\
            '老化指數','戶數',"人口數",'男性人口數','女性人口數',\
            '國小數量','國小學生數','國中數量','國中學生數量','高中數量',\
            '高中學生數量','納稅單位', '綜合所得總額', '所得平均數','所得中位數', \
            '所得第一分位數', '所得第三分位數','所得標準差', '所得變異係數','廟宇數量',\
            '台鐵火車站站數','台鐵人流', "醫學中心+區域醫院+地區醫院",'診所']
col = ['VILLCODE', 'COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'Cluster',
       'high_rat', 'mid_rat', 'low_rat', 'high_edu', 'mid_edu', 'low_edu',
       'Hospital_1', 'Hospital_2', 'Hospital_3', 'Hospital_4',
       'M_F_RAT', 'P_H_CNT', 'P_DEN', 'DEPENDENCY', 'YOUN_DEP', 'OLD_DEP', 
       'AGING_IDX', 'H_CNT',  'P_CNT', 'M_CNT', 'F_CNT', 'Ele', 
       'Ele_stu', 'Junior', 'Junior_stu', 'Senior', 'Senior_stu', 
       'TaxPayer', 'Income_total','Income_mean', 'Income_median', 'Income_Q1',
       'Income_Q3', 'Income_stande', 'Income_CV',  'Temple', 'Train-stat',
       'Train-crow','Hospital', 'Clinic']

x = zip(col, col_explain)
codebook = dict(x)
print(codebook)

with open('..\\..\\Roaming-Dogs-Data\\@Taiwan_village_variable\\CodeBook.txt', 'w') as f: 
    for key, value in codebook.items(): 
        f.write('%s:%s\n' % (key, value))
with open('..\\..\\Roaming-Dogs-Data\\@Taiwan_sampling_village_variable\\CodeBook.txt', 'w') as f: 
    for key, value in codebook.items(): 
        f.write('%s:%s\n' % (key, value))

{'VILLCODE': '縣市名稱', 'COUNTYNAME': '鄉鎮市區名稱', 'TOWNNAME': '村里名稱', 'VILLNAME': '村里英文', 'VILLENG': '縣市ID', 'COUNTYID': '縣市代碼', 'COUNTYCODE': '鄉鎮市區ID', 'TOWNID': '鄉鎮市區代碼', 'TOWNCODE': '村里代碼', 'NOTE': '備註', 'Cluster': '分群', 'high_rat': '高等教育人口比', 'mid_rat': '中等教育人口比', 'low_rat': '初等教育人口比', 'high_edu': '高等教育人口', 'mid_edu': '中等教育人口', 'low_edu': '初等教育人口', 'Hospital_1': '醫學中心', 'Hospital_2': '區域醫院', 'Hospital_3': '地區醫院', 'Hospital_4': '診所', 'M_F_RAT': '性比例', 'P_H_CNT': '戶量', 'P_DEN': '人口密度', 'DEPENDENCY': '扶養比', 'YOUN_DEP': '扶幼比', 'OLD_DEP': '扶老比', 'AGING_IDX': '老化指數', 'H_CNT': '戶數', 'P_CNT': '人口數', 'M_CNT': '男性人口數', 'F_CNT': '女性人口數', 'Ele': '國小數量', 'Ele_stu': '國小學生數', 'Junior': '國中數量', 'Junior_stu': '國中學生數量', 'Senior': '高中數量', 'Senior_stu': '高中學生數量', 'TaxPayer': '納稅單位', 'Income_total': '綜合所得總額', 'Income_mean': '所得平均數', 'Income_median': '所得中位數', 'Income_Q1': '所得第一分位數', 'Income_Q3': '所得第三分位數', 'Income_stande': '所得標準差', 'Income_CV': '所得變異係數', 'Temple': '廟宇數量', 'Train-stat': '台鐵火車站站數', 'Train-crow

In [32]:
Taiwan_village_.to_file(driver='ESRI Shapefile', \
    filename='..\\..\\Roaming-Dogs-Data\@Taiwan_village_variable\\Taiwan_village_variable.shp', encoding="Big5")


### Sampling villages 
    經整理有 628 里

In [19]:
path ='..\\..\\Roaming-Dogs-Data\\zz_期末報告\\'
N = gpd.read_file(path+'1125推估結果.csv' ,encoding="big5")
Taiwan_village_variable = gpd.read_file("..\\..\\Roaming-Dogs-Data\\@Taiwan_village_variable\\Taiwan_village_variable.shp",encoding="big5")
Variable_XY = Taiwan_village_variable.merge(N, left_on = ['COUNTYNAME','TOWNNAME','VILLNAME'], right_on = ['country','town', 'village'], how = 'right')
idx = (Variable_XY["TOWNNAME"].isnull())
print(Variable_XY[idx][['country', 'town', 'village','Nt']].sort_values('town'))

    country town   village   Nt
575     臺南市  中西區       進學里    1
574     臺南市  中西區       五妃里    2
323     臺南市   北區       華興里    8
105     臺南市  北門區  蚵寮里(鯤江里)  152
139     臺南市  安南區       顯宮里   30
213     臺南市  安定區       保西里   13
69      臺南市  安定區       大同里   22
35      臺南市  官田區       西庄里   26
37      臺南市  官田區       拔林里   38
11      臺南市  將軍區       仁和里   52
597     屏東縣  屏東市       永昌里    5
317     屏東縣  屏東市       萬年里    4
615     彰化縣  彰化市       南瑤里    9
595     臺南市  永康區       鹽洲里   12
408     臺東縣  臺東市       鐵花里    7
181     臺東縣  臺東市       強國里   16
89      臺東縣  臺東市       建業里   27


C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


1. 進學里：由大南里、**進學里**進行整編，大南門城、南門公園位於本里，以「南門里」為新里名。
2. 五妃里： 改名為郡王里
3. 華興里：國興里、**華興里**（部分）進行整編，二里合併為合興里(107 年 )
4. 顯宮里：更名為媽祖宮里(2021/04)
5. 保西里：安加里、**保西里**進行整編，二里合為安加里(107 年)
6. 大同里：**大同里**、六嘉里進行整編，二里合為嘉同里(107 年)
7. 西庄里：**西庄里**、東庄里進行整編，二里合為東西庄里(107 年)
8. 拔林里：**拔林里**、渡頭里進行整編，二里合為渡拔里(107 年)
9. 仁和里：找不到
10. 永昌里：永光、永順、**永昌**，3里合併為永光里
11. 萬年里：一心里（一心、萬年，2里合併）
12. 南瑤里：疆界shp檔里名為"南?里"
13. 鹽洲里：疆界shp檔里名為 ?洲里
14~16. 鐵花里、強國里、建業里：會重新整合合併 (http://suggest.taitungcity.gov.tw/village/qna.htm)



In [33]:
path ='..\\..\\Roaming-Dogs-Data\\zz_期末報告\\'
N = pd.read_csv(path+'1125推估結果_02.csv' ,encoding="big5")
Taiwan_village_variable = gpd.read_file("..\\..\\Roaming-Dogs-Data\\@Taiwan_village_variable\\Taiwan_village_variable.shp",encoding="big5")

C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [34]:
Variable_XY = Taiwan_village_variable.merge(N[['country','town', 'village','Nt']], left_on = ['COUNTYNAME','TOWNNAME','VILLNAME'], right_on = ['country','town', 'village'], how = 'inner')

In [35]:
print(Variable_XY.shape)
Variable_XY = gpd.GeoDataFrame(Variable_XY, geometry = 'geometry', crs =Taiwan_village_variable.crs )


(628, 57)


In [36]:

Variable_XY.to_file(driver='ESRI Shapefile', \
    filename='..\\..\\Roaming-Dogs-Data\\@Taiwan_sampling_village_variable\\Taiwan_sampling_village_variable.shp', encoding="Big5")